In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
import numpy as np
import pandas as pd
import time
# from base_func import remain_feat
start_time=time.time()
# 读取数据
train=pd.read_csv('D:\\1python notes\\aliyun\\new\\meinian_round1_train_20180408.csv',sep=',',encoding='gbk')
test=pd.read_csv('D:\\1python notes\\aliyun\\new\\meinian_round1_test_b_20180505.csv',sep=',',encoding='gbk')
data_part1=pd.read_csv('D:\\1python notes\\aliyun\\new\\meinian_round1_data_part1_20180408.txt',sep='$',encoding='utf-8')
data_part2=pd.read_csv('D:\\1python notes\\aliyun\\new\\meinian_round1_data_part2_20180408.txt',sep='$',encoding='utf-8')
# train=pd.read_csv('D:\\1python notes\\aliyun\\new\\meinian_round1_train_20180408.csv',sep=',',encoding='gbk')

# data_part1和data_part2进行合并，并剔除掉与train、test不相关vid所在的行
part1_2 = pd.concat([data_part1,data_part2],axis=0)#{0/'index', 1/'columns'}, default 0
part1_2 = pd.DataFrame(part1_2).sort_values('vid').reset_index(drop=True)
vid_set=pd.concat([train['vid'],test['vid']],axis=0)
vid_set=pd.DataFrame(vid_set).sort_values('vid').reset_index(drop=True)
part1_2=part1_2[part1_2['vid'].isin(vid_set['vid'])]

# 根据常识判断无用的'检查项'table_id，过滤掉无用的table_id
def filter_None(data):
    data=data[data['field_results']!='']
    data=data[data['field_results']!='未查']
    return data

part1_2=filter_None(part1_2)

# 过滤列表，过滤掉不重要的table_id 所在行
filter_list=['0203','0209','0702','0703','0705','0706','0709','0726','0730','0731','3601',
             '1308','1316']

part1_2=part1_2[~part1_2['table_id'].isin(filter_list)]

# 重复数据的拼接操作
def merge_table(df):
    df['field_results'] = df['field_results'].astype(str)
    if df.shape[0] > 1:
        merge_df = " ".join(list(df['field_results']))
    else:
        merge_df = df['field_results'].values[0]
    return merge_df

# 数据简单处理
print(part1_2.shape)
vid_tabid_group = part1_2.groupby(['vid','table_id']).size().reset_index()
# print(vid_tabid_group.head())
# print(vid_tabid_group.shape)
#                      vid               table_id  0
# 0  000330ad1f424114719b7525f400660b     0101     1
# 1  000330ad1f424114719b7525f400660b     0102     3

# 重塑index用来去重,区分重复部分和唯一部分
print('------------------------------去重和组合-----------------------------')
vid_tabid_group['new_index'] = vid_tabid_group['vid'] + '_' + vid_tabid_group['table_id']
vid_tabid_group_dup = vid_tabid_group[vid_tabid_group[0]>1]['new_index']

# print(vid_tabid_group_dup.head()) #000330ad1f424114719b7525f400660b_0102
part1_2['new_index'] = part1_2['vid'] + '_' + part1_2['table_id']

dup_part = part1_2[part1_2['new_index'].isin(list(vid_tabid_group_dup))]
dup_part = dup_part.sort_values(['vid','table_id'])
unique_part = part1_2[~part1_2['new_index'].isin(list(vid_tabid_group_dup))]

part1_2_dup = dup_part.groupby(['vid','table_id']).apply(merge_table).reset_index()
part1_2_dup.rename(columns={0:'field_results'},inplace=True)
part1_2_res = pd.concat([part1_2_dup,unique_part[['vid','table_id','field_results']]])

table_id_group=part1_2.groupby('table_id').size().sort_values(ascending=False)
table_id_group.to_csv('D:\\1python notes\\aliyun\\new\\temp\\part_tabid_size.csv',encoding='utf-8')

# 行列转换
print('--------------------------重新组织index和columns---------------------------')
merge_part1_2 = part1_2_res.pivot(index='vid',values='field_results',columns='table_id')
print('--------------新的part1_2组合完毕----------')
print(merge_part1_2.shape)
merge_part1_2.to_csv('D:\\1python notes\\aliyun\\new\\temp\\merge_part1_2.csv',encoding='utf-8')
print(merge_part1_2.head())
del merge_part1_2

time.sleep(10)
print('------------------------重新读取数据merge_part1_2--------------------------')
merge_part1_2=pd.read_csv('D:\\1python notes\\aliyun\\new\\temp\\merge_part1_2.csv',sep=',',encoding='utf-8')

# 删除掉一些出现次数低，缺失比例大的字段，保留超过阈值的特征
def remain_feat(df,thresh=0.9):
    exclude_feats = []
    print('----------移除数据缺失多的字段-----------')
    print('移除之前总的字段数量',len(df.columns))
    num_rows = df.shape[0]
    for c in df.columns:
        num_missing = df[c].isnull().sum()
        if num_missing == 0:
            continue
        missing_percent = num_missing / float(num_rows)
        if missing_percent > thresh:
            exclude_feats.append(c)
    print("移除缺失数据的字段数量: %s" % len(exclude_feats))
    # 保留超过阈值的特征
    feats = []
    for c in df.columns:
        if c not in exclude_feats:
            feats.append(c)
    print('剩余的字段数量',len(feats))
    return feats
feats=remain_feat(merge_part1_2,thresh=0.96)


merge_part1_2=merge_part1_2[feats]
merge_part1_2.to_csv('D:\\1python notes\\aliyun\\new\\temp\\merge_part1_2.csv')

# 找到train，test各自属性进行拼接
train_of_part=merge_part1_2[merge_part1_2['vid'].isin(train['vid'])]
test_of_part=merge_part1_2[merge_part1_2['vid'].isin(test['vid'])]

train=pd.merge(train,train_of_part,on='vid')
test=pd.merge(test,test_of_part,on='vid')

# 清洗训练集中的五个指标
def clean_label(x):
    x=str(x)
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
    return x

# 数据清洗
def data_clean(df):
    for c in ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白']:
        df[c]=df[c].apply(clean_label)
        df[c]=df[c].astype('float64')
    return df
train=data_clean(train)

print('---------------保存train_set和test_set---------------------')
train.to_csv('D:\\1python notes\\aliyun\\new\\temp\\train_set.csv',index=False,encoding='utf-8')
test.to_csv('D:\\1python notes\\aliyun\\new\\temp\\test_set.csv',index=False,encoding='utf-8')

end_time=time.time()
print('程序总共耗时:%d 秒'%int(end_time-start_time))


'\nimport numpy as np\nimport pandas as pd\nimport time\n# from base_func import remain_feat\nstart_time=time.time()\n# 读取数据\ntrain=pd.read_csv(\'D:\\1python notes\\aliyun\\new\\meinian_round1_train_20180408.csv\',sep=\',\',encoding=\'gbk\')\ntest=pd.read_csv(\'D:\\1python notes\\aliyun\\new\\meinian_round1_test_b_20180505.csv\',sep=\',\',encoding=\'gbk\')\ndata_part1=pd.read_csv(\'D:\\1python notes\\aliyun\\new\\meinian_round1_data_part1_20180408.txt\',sep=\'$\',encoding=\'utf-8\')\ndata_part2=pd.read_csv(\'D:\\1python notes\\aliyun\\new\\meinian_round1_data_part2_20180408.txt\',sep=\'$\',encoding=\'utf-8\')\n# train=pd.read_csv(\'D:\\1python notes\\aliyun\\new\\meinian_round1_train_20180408.csv\',sep=\',\',encoding=\'gbk\')\n\n# data_part1和data_part2进行合并，并剔除掉与train、test不相关vid所在的行\npart1_2 = pd.concat([data_part1,data_part2],axis=0)#{0/\'index\', 1/\'columns\'}, default 0\npart1_2 = pd.DataFrame(part1_2).sort_values(\'vid\').reset_index(drop=True)\nvid_set=pd.concat([train[\'vid\'],te

In [5]:
test_data=pd.read_csv('D:\\1python notes\\aliyun\\new\\temp\\test_set.csv',encoding='utf-8')

train_data=pd.read_csv('D:\\1python notes\\aliyun\\new\\temp\\train_data.csv',encoding='utf-8')

C:\Users\lk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (9,104,107,108,109,110,111,113,121,123,126,151,156,157,158,159,160,161,164,165,166,167,168,179,183,190,193,198,205,218,229,230,235,238,243,248,271,273,296,297,301,305,323,324,358,359,360,361,362,364,365,366,368,370,371,372,373,374,375,376,377,378,379,380) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\lk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (10,47,105,108,109,110,111,112,113,114,119,120,121,122,123,124,127,146,150,152,153,154,155,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,180,181,182,183,184,185,189,190,191,192,193,194,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,221,222,226,227,228,229,230,231,232,235,236,239,240,241,244,245,246,247,253,254,266,271,272,274,287,288,

In [6]:
train_data.head()

,Unnamed: 0,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0101,0102,...,979026,979027,A201,A202,A301,A302,A601,A701,A703,A705
0,0,002d1e4859fafd9ded2a2e1e7c839b62,165.0,100.0,2.08,1.29,3.24,NaN,双侧颈部各动脉内径正常，内膜-中膜薄而平滑，连续性好，管腔内未见异常回声，彩色血流充盈良好，...,"甲状腺右叶结节甲状腺左叶囊实性结节,建议定期检查左侧颈部淋巴结肿大 肝、胆、胰、脾、左肾、右...",...,NaN,NaN,两侧胸廓对称，两肺纹理增粗，右肺上叶见片絮状模糊影，并见有小圆形高密度影。边缘较清晰。余...,炎性病变可能右肺钙化灶,双侧大脑半球对称，大脑灰白质形态、位置、信号未见异常。脑干、小脑实质未见明显异常信号，FLA...,头部MRI未发现明显异常,具体内容请见分析报告。,4.8,229,数字化肝超未发现明显异常
1,1,92dd479df5e30ab6a0a1cf85ac53efc3,141.0,97.0,2.64,1.36,4.75,NaN,甲状腺形态大小正常，边界清晰，内部回声分布均匀，未见明显异常回声。 双侧颈总动脉管径对称，内...,前列腺未发现明显异常 甲状腺彩超未发现明显异常 膀胱未发现明显异常 颈动脉彩超未发现明显异常...,...,NaN,NaN,胸廓对称，双肺纹理清晰，走行自然，未见异常实变影，双肺门不大。纵隔窗示纵隔无偏移，心影及大血...,肺部CT扫描未发现明显异常,NaN,NaN,具体内容请见分析报告。,7.1,315,脂肪衰减值大于240，肝脏脂肪含量超过正常值。
2,2,6bb59d517c4c70f8f50844d24fbd0355,120.0,80.0,1.37,1.25,2.66,NaN,各房室腔大小正常，主动脉主波波幅正常，重博波存在，室间隔，左室后壁无增厚，活动正常，未见节段...,心脏彩超未发现明显异常 胃粘膜层毛糙 肝:脂肪肝（中度）胆、胰、脾未发现明显异常 颈动脉彩超...,...,NaN,NaN,胸廓对称，双肺纹理清晰，走行自然，未见异常实变影，双肺门不大。纵隔窗示纵隔无偏移，心影及大血...,肺部CT（不出片）未发现明显异常,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0ebb42adae512906f7e1135da734ea63,100.0,70.0,1.27,2.21,1.73,NaN,双侧乳腺层次结构清晰，内部回声分布均匀，未见明显异常回声。 双侧颈总动脉形态结构正常，管壁不...,双乳腺彩超未发现明显异常 心脏彩超未发现明显异常 肝、胆、脾未发现明显异常 颈动脉彩超（双侧...,...,NaN,NaN,右上肺可见索条及钙化影 两侧大脑半球，小脑及脑干形态正常，未见明显异常密度病灶，两侧脑室，第...,陈旧性肺结核 头部CT（不出片）未发现明显异常,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ebe7811e919109c42c092abbd98b4ca6,110.0,80.0,0.80,1.87,2.21,NaN,各房室腔大小正常，主动脉主波波形正常，重搏波存在，室间隔，左室后壁无增厚，活动正常，未见节段...,双侧乳腺轻度增生 双侧颈动脉内-中膜毛糙 心脏彩超未发现明显异常 甲状腺右叶结节 胆:胆囊壁...,...,NaN,NaN,颈椎生理曲线僵直。颈椎椎体及相应椎小关节骨质增生，颈椎间盘突出。 双肺下叶见索条影，双肺门不...,肺陈旧性病变 颈椎病,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train_data

,Unnamed: 0,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0101,0102,...,979026,979027,A201,A202,A301,A302,A601,A701,A703,A705
0,0,002d1e4859fafd9ded2a2e1e7c839b62,165.0,100.0,2.08,1.29,3.24,NaN,双侧颈部各动脉内径正常，内膜-中膜薄而平滑，连续性好，管腔内未见异常回声，彩色血流充盈良好，...,"甲状腺右叶结节甲状腺左叶囊实性结节,建议定期检查左侧颈部淋巴结肿大 肝、胆、胰、脾、左肾、右...",...,NaN,NaN,两侧胸廓对称，两肺纹理增粗，右肺上叶见片絮状模糊影，并见有小圆形高密度影。边缘较清晰。余...,炎性病变可能右肺钙化灶,双侧大脑半球对称，大脑灰白质形态、位置、信号未见异常。脑干、小脑实质未见明显异常信号，FLA...,头部MRI未发现明显异常,具体内容请见分析报告。,4.8,229,数字化肝超未发现明显异常
1,1,92dd479df5e30ab6a0a1cf85ac53efc3,141.0,97.0,2.64,1.36,4.75,NaN,甲状腺形态大小正常，边界清晰，内部回声分布均匀，未见明显异常回声。 双侧颈总动脉管径对称，内...,前列腺未发现明显异常 甲状腺彩超未发现明显异常 膀胱未发现明显异常 颈动脉彩超未发现明显异常...,...,NaN,NaN,胸廓对称，双肺纹理清晰，走行自然，未见异常实变影，双肺门不大。纵隔窗示纵隔无偏移，心影及大血...,肺部CT扫描未发现明显异常,NaN,NaN,具体内容请见分析报告。,7.1,315,脂肪衰减值大于240，肝脏脂肪含量超过正常值。
2,2,6bb59d517c4c70f8f50844d24fbd0355,120.0,80.0,1.37,1.25,2.66,NaN,各房室腔大小正常，主动脉主波波幅正常，重博波存在，室间隔，左室后壁无增厚，活动正常，未见节段...,心脏彩超未发现明显异常 胃粘膜层毛糙 肝:脂肪肝（中度）胆、胰、脾未发现明显异常 颈动脉彩超...,...,NaN,NaN,胸廓对称，双肺纹理清晰，走行自然，未见异常实变影，双肺门不大。纵隔窗示纵隔无偏移，心影及大血...,肺部CT（不出片）未发现明显异常,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0ebb42adae512906f7e1135da734ea63,100.0,70.0,1.27,2.21,1.73,NaN,双侧乳腺层次结构清晰，内部回声分布均匀，未见明显异常回声。 双侧颈总动脉形态结构正常，管壁不...,双乳腺彩超未发现明显异常 心脏彩超未发现明显异常 肝、胆、脾未发现明显异常 颈动脉彩超（双侧...,...,NaN,NaN,右上肺可见索条及钙化影 两侧大脑半球，小脑及脑干形态正常，未见明显异常密度病灶，两侧脑室，第...,陈旧性肺结核 头部CT（不出片）未发现明显异常,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ebe7811e919109c42c092abbd98b4ca6,110.0,80.0,0.80,1.87,2.21,NaN,各房室腔大小正常，主动脉主波波形正常，重搏波存在，室间隔，左室后壁无增厚，活动正常，未见节段...,双侧乳腺轻度增生 双侧颈动脉内-中膜毛糙 心脏彩超未发现明显异常 甲状腺右叶结节 胆:胆囊壁...,...,NaN,NaN,颈椎生理曲线僵直。颈椎椎体及相应椎小关节骨质增生，颈椎间盘突出。 双肺下叶见索条影，双肺门不...,肺陈旧性病变 颈椎病,NaN,NaN,NaN,NaN,NaN,NaN
5,5,c2690ddf171a3de695dced12fd93c1d1,110.0,80.0,1.40,1.74,2.09,NaN,甲状腺形态大小正常，腺体回声均匀，左叶可见大小约2x2mm低回声。,肝、胆、脾未发现明显异常 甲状腺左叶结节 前列腺:前列腺钙化灶左肾、右肾、膀胱未发现明显异常,...,NaN,NaN,双肺见索条影，双肺门不大。纵隔窗示纵隔无偏移，心影及大血管形态正常，纵隔内未见肿块及肿大淋巴...,肺陈旧性病变,NaN,NaN,NaN,NaN,NaN,NaN
6,6,d7017fda4e550b14008c730854b89b95,130.0,78.0,2.50,1.22,4.53,NaN,升主动脉内径：20mm 左房前后径：23mm 右室流出道：22mm ...,前列腺:前列腺钙化灶左肾、右肾、膀胱未发现明显异常 肝、胆、胰、脾未发现明显异常 心脏彩超未...,...,NaN,NaN,NaN,NaN,NaN,NaN,详见纸质报告 详见纸质报告 详见纸质报告,NaN,NaN,NaN
7,7,d6b92248a262a491d6250dcebc103e71,111.0,71.0,3.36,1.08,2.88,NaN,NaN,肝:脂肪肝（中度）胆、胰、脾、左肾、右肾未发现明显异常 前列腺:前列腺钙化灶膀胱未发现明显异常,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,345742c522a571dbb0483474242d3484,94.0,64.0,0.62,1.34,1.67,NaN,"双侧乳腺腺体层增厚,内部结构紊乱,回声不均，未见异常血流。左侧乳腺腺体层内见一低回声团，测值...",左附件:多囊卵巢子宫、右附件未发现明显异常 肝、胆、胰、脾、左肾、右肾未发现明显异常 双侧乳...,...,NaN,NaN,胸廓对称，双肺纹理清晰，走行自然，未见异常实变影，双肺门不大。纵隔窗示纵隔无偏移，心影及大血...,CT胸部未发现明显异常,NaN,NaN,NaN,NaN,NaN,NaN
9,9,c00413e3df904256278f6938ab55ab7a,164.0,98.0,1.12,1.51,2.79,NaN,双侧乳腺轮廓清晰，层次清楚，腺体无增厚，分布均匀，腺管无扩张。CDFI：内未见明显异常血流。,胆囊壁毛糙胆总管增宽左肾:左肾囊肿左肾结石肝、胰、脾、右肾未发现明显异常 乳腺彩超未发现明显异常,...,NaN,NaN,详见纸质报告,详见纸质报告,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
train_data.describe()

,Unnamed: 0,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0105,0106,0108,...,809057,809058,809059,809060,809061,979010,979024,979025,979026,979027
count,38199.000000,38192.000000,38192.000000,38199.000000,38199.000000,38199.000000,3334.000000,1728.000000,1720.000000,1635.000000,...,3302.000000,3302.000000,3302.000000,3166.000000,3166.000000,2561.000000,2722.000000,2611.000000,2251.000000,2251.000000
mean,19099.000000,126.049618,79.642229,1.616563,1.406683,2.769720,15.446761,29.643837,8.940698,44.859443,...,8.385251,11.211993,0.175633,20.647075,38.988945,43.808266,6.483575,3.607239,0.000431,0.015460
std,11027.245803,19.275625,512.318203,1.344201,0.341184,0.852187,4.748890,4.203795,1.734424,5.069207,...,1.274535,2.255826,0.044340,8.450500,15.522624,4.759779,17.229730,9.676132,0.002117,0.038309
min,0.000000,0.000000,0.000000,0.100000,0.150000,-1.200000,4.540000,2.940000,1.380000,3.120000,...,5.300000,2.500000,0.050000,3.260000,6.000000,19.700000,0.000000,0.000000,0.000000,0.000000
25%,9549.500000,112.000000,68.000000,0.880000,1.160000,2.180000,11.260000,27.000000,8.000000,42.000000,...,7.500000,9.700000,0.150000,15.252500,29.000000,40.400000,0.000000,0.000000,0.000000,0.000000
50%,19099.000000,124.000000,76.000000,1.270000,1.350000,2.690000,16.010000,29.000000,9.000000,45.000000,...,8.200000,10.800000,0.170000,20.835000,39.000000,43.900000,0.010000,0.100000,0.000000,0.000000
75%,28648.500000,137.000000,85.000000,1.910000,1.600000,3.260000,18.730000,32.000000,10.000000,48.000000,...,9.100000,12.400000,0.200000,25.490000,48.000000,47.300000,0.010000,0.200000,0.000000,0.000000
max,38198.000000,252.000000,100164.000000,28.800000,4.780000,11.500000,39.610000,54.000000,43.000000,68.000000,...,16.900000,30.000000,0.510000,89.260000,161.000000,66.900000,105.000000,99.000000,0.020000,0.400000


In [9]:
cols=train_data.columns

In [10]:
Col=[]
for i in cols:
    if train_data[i].count()>382:
        Col.append(i)
len(Col)       

394

In [11]:
# 清洗训练集中的五个指标
def clean_label(x):
    x=str(x)
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
    return x

text_col=[]
col3=train_data.columns
# 数据清洗
def data_clean(df):
    for c in col3:
        try: 
            df[c]=df[c].apply(clean_label)
            df[c]=df[c].astype('float64')
        except:
            text_col.append(c)
    return df

train=data_clean(train_data)


# def data_clean(df):
#     for c in ['A701']:
        
#         df[c]=df[c].apply(clean_label)
#         df[c]=df[c].astype('float64')
        
            
#     return df

# train=data_clean(train_data)

In [12]:
train

,Unnamed: 0,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0101,0102,...,979026,979027,A201,A202,A301,A302,A601,A701,A703,A705
0,0.0,002d,165.0,100.0,2.08,1.29,3.24,NaN,双侧颈部,甲状腺右,...,NaN,NaN,两侧,炎性病变,双侧大脑,头部MR,具体内容,4.8,229.,数字化肝
1,1.0,92dd,141.0,97.0,2.64,1.36,4.75,NaN,甲状腺形,前列腺未,...,NaN,NaN,胸廓对称,肺部CT,nan,nan,具体内容,7.1,315.,脂肪衰减
2,2.0,6bb5,120.0,80.0,1.37,1.25,2.66,NaN,A峰。C,心脏彩超,...,NaN,NaN,胸廓对称,肺部CT,nan,nan,nan,nan,nan,nan
3,3.0,0ebb,100.0,70.0,1.27,2.21,1.73,NaN,A峰。C,双乳腺彩,...,NaN,NaN,右上肺可,陈旧性肺,nan,nan,nan,nan,nan,nan
4,4.0,ebe7,110.0,80.0,0.80,1.87,2.21,NaN,A峰。C,双侧乳腺,...,NaN,NaN,颈椎生理,肺陈旧性,nan,nan,nan,nan,nan,nan
5,5.0,c269,110.0,80.0,1.40,1.74,2.09,NaN,甲状腺形,肝、胆、,...,NaN,NaN,双肺见索,肺陈旧性,nan,nan,nan,nan,nan,nan
6,6.0,d701,130.0,78.0,2.50,1.22,4.53,NaN,升主动脉,前列腺:,...,NaN,NaN,nan,nan,nan,nan,详见纸质,nan,nan,nan
7,7.0,d6b9,111.0,71.0,3.36,1.08,2.88,NaN,nan,肝:脂肪,...,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan
8,8.0,3457,94.0,64.0,0.62,1.34,1.67,NaN,双侧乳腺,左附件:,...,NaN,NaN,胸廓对称,CT胸部,nan,nan,nan,nan,nan,nan
9,9.0,c004,164.0,98.0,1.12,1.51,2.79,NaN,双侧乳腺,胆囊壁毛,...,NaN,NaN,详见纸质,详见纸质,nan,nan,nan,nan,nan,nan


In [13]:
train.describe()

,Unnamed: 0,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0105,0106,0108,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
count,38199.000000,38192.000000,38192.000000,38199.000000,38199.000000,38199.000000,3334.000000,1728.000000,1720.000000,1635.000000,...,6766.000000,6766.000000,6870.000000,6871.000000,6870.000000,6870.000000,2722.000000,2611.000000,2251.000000,2251.000000
mean,19099.000000,126.049618,77.045795,1.616563,1.406683,2.769720,15.408500,29.643837,8.940698,44.859443,...,0.547857,2.497731,56.978355,5.831535,34.168777,6.119066,6.481198,3.607239,0.000431,0.015460
std,11027.245803,19.275625,14.201342,1.344201,0.341184,0.852187,4.739637,4.203795,1.734424,5.069207,...,0.337723,2.095664,7.925242,1.565357,7.411398,1.567620,17.228270,9.676132,0.002117,0.038309
min,0.000000,0.000000,0.000000,0.100000,0.150000,-1.200000,4.540000,2.940000,1.380000,3.120000,...,0.000000,0.000000,28.000000,0.600000,5.700000,1.710000,0.000000,0.000000,0.000000,0.000000
25%,9549.500000,112.000000,68.000000,0.880000,1.160000,2.180000,11.200000,27.000000,8.000000,42.000000,...,0.300000,1.200000,51.800000,4.800000,29.200000,5.050000,0.000000,0.000000,0.000000,0.000000
50%,19099.000000,124.000000,76.000000,1.270000,1.350000,2.690000,16.000000,29.000000,9.000000,45.000000,...,0.500000,1.940000,57.100000,5.600000,34.000000,5.890000,0.010000,0.100000,0.000000,0.000000
75%,28648.500000,137.000000,85.000000,1.910000,1.600000,3.260000,18.700000,32.000000,10.000000,48.000000,...,0.700000,3.100000,62.200000,6.700000,38.900000,6.960000,0.010000,0.200000,0.000000,0.000000
max,38198.000000,252.000000,1001.000000,28.800000,4.780000,11.500000,39.600000,54.000000,43.000000,68.000000,...,2.800000,40.500000,89.000000,15.900000,64.600000,19.500000,105.000000,99.000000,0.020000,0.400000


In [14]:
def getUsefulColumns(columns):
    filteredColumns=[]
    for clumn in columns:
        if(train[clumn].dtypes == 'float64'):
            filteredColumns.append(clumn)
    return filteredColumns

columns=train.columns
l4=getUsefulColumns(columns)
len(l4)

158

In [15]:
'''
测试集 test data
'''

'\n测试集\n'

In [16]:
# 清洗训练集中的五个指标
def clean_label(x):
    x=str(x)
    if ' ' in x: #49 4
        i=x.index(' ')
        x=x[0:i]
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
    return x

text_col=[]
col4=test_data.columns
# 数据清洗
def data_clean(df):
    for c in col4:
        try: 
            df[c]=df[c].apply(clean_label)
            df[c]=df[c].astype('float64')
        except:
            text_col.append(c)
    return df

test=data_clean(test_data)
test

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0101,0102,0104,...,979026,979027,A201,A202,A301,A302,A601,A701,A703,A705
0,001f,NaN,NaN,NaN,NaN,NaN,NaN,甲状腺形,甲状腺彩,nan,...,NaN,NaN,详见纸质,胸部CT,nan,nan,详见纸质,6.4,nan,数字化肝
1,0029,NaN,NaN,NaN,NaN,NaN,12.80,甲状腺形,右肾:右,nan,...,NaN,NaN,肺窗示：,肺纤维灶,nan,nan,nan,nan,nan,nan
2,002f,NaN,NaN,NaN,NaN,NaN,NaN,甲状腺形,肝、胆、,nan,...,NaN,NaN,nan,nan,nan,nan,nan,6.8,227.,数字化肝
3,0037,NaN,NaN,NaN,NaN,NaN,NaN,右侧乳腺,右侧乳腺,nan,...,NaN,NaN,胸廓对称,胸部CT,nan,nan,详见纸质,nan,nan,nan
4,0039,NaN,NaN,NaN,NaN,NaN,NaN,双侧乳腺,双侧乳腺,nan,...,NaN,NaN,胸廓对称,CT检查,nan,nan,nan,nan,nan,nan
5,0043,NaN,NaN,NaN,NaN,NaN,NaN,双侧甲状,肝、胆、,nan,...,NaN,NaN,nan,nan,详见纸质,详见纸质,nan,12.1,215.,经数字化
6,0047,NaN,NaN,NaN,NaN,NaN,NaN,nan,前列腺:,nan,...,NaN,NaN,两侧大脑,详见纸质,nan,nan,详见纸质,nan,nan,nan
7,0048,NaN,NaN,NaN,NaN,NaN,NaN,双侧甲状,甲状腺左,nan,...,NaN,NaN,胸廓对称,胸部CT,nan,nan,nan,nan,nan,nan
8,0050,NaN,NaN,NaN,NaN,NaN,NaN,双侧甲状,甲状腺双,nan,...,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan
9,0051,NaN,NaN,NaN,NaN,NaN,NaN,甲状腺形,肝:脂肪,nan,...,NaN,NaN,nan,nan,nan,nan,详见纸质,4.4,265.,脂肪衰减


In [17]:
def getUsefulColumns(columns):
    filteredColumns=[]
    for clumn in columns:
        if(test[clumn].dtypes == 'float64'):
            filteredColumns.append(clumn)
    return filteredColumns

columns=test.columns
l5=getUsefulColumns(columns)
len(l5)

194

In [20]:
len(l4)

158

In [21]:
test.loc[4870, '2404']=np.nan
test['2404']=test['2404'].astype('float64')
test['2404']

0       169.0
1       153.0
2       173.0
3       163.0
4       148.0
5       160.0
6       160.0
7       175.0
8       173.0
9       171.0
10      168.0
11      169.0
12      154.0
13      160.0
14      173.0
15      157.0
16      160.0
17      172.0
18      154.0
19      168.0
20      164.0
21      176.0
22      176.0
23      156.0
24      162.0
25      175.0
26      161.0
27      171.0
28      154.0
29      168.0
        ...  
9502    167.0
9503    161.0
9504    166.0
9505    157.0
9506    164.0
9507    169.0
9508    172.0
9509    158.0
9510    178.0
9511    153.0
9512    178.0
9513    164.0
9514    160.0
9515    151.0
9516    154.0
9517    170.0
9518    167.0
9519    173.0
9520    168.0
9521    152.0
9522    162.0
9523    159.0
9524    165.0
9525    151.0
9526    159.0
9527    162.0
9528    171.0
9529    174.0
9530    175.0
9531    173.0
Name: 2404, Length: 9532, dtype: float64

In [37]:
# filteredColumns=getUsefulColumns(columns)
filteredColumns=getUsefulColumns(columns)

useful_train=train_data[l4]
useful_train=useful_train.drop('Unnamed: 0', axis=1)
# filteredColumns=usef
useful_train

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0105,0106,0108,0109,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
0,165.0,100.0,2.08,1.29,3.24,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141.0,97.0,2.64,1.36,4.75,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120.0,80.0,1.37,1.25,2.66,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100.0,70.0,1.27,2.21,1.73,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110.0,80.0,0.80,1.87,2.21,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,110.0,80.0,1.40,1.74,2.09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,130.0,78.0,2.50,1.22,4.53,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,111.0,71.0,3.36,1.08,2.88,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,94.0,64.0,0.62,1.34,1.67,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,164.0,98.0,1.12,1.51,2.79,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# 过滤标签中小于0的值所在行
# 过滤掉标签中的空值所在行
# 过滤掉舒张压过大的
def dropNa(df,columns):
    for column in columns:
        df=df[df[column].notnull()]
    return df

columns=useful_train.columns
filtered_train=useful_train[useful_train[columns[4]]>0]
filtered_train=filtered_train[filtered_train[columns[1]]<790]
filtered_train=dropNa(filtered_train,columns[0:5])
filtered_train.describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0105,0106,0108,0109,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
count,38187.000000,38187.000000,38187.000000,38187.000000,38187.000000,3334.000000,1726.000000,1718.000000,1634.000000,1664.000000,...,6762.000000,6762.000000,6866.000000,6867.000000,6866.000000,6866.000000,2720.000000,2609.000000,2250.000000,2250.000000
mean,126.047765,76.998403,1.616139,1.406649,2.769914,15.408500,29.641686,8.938882,44.859969,8.605938,...,0.547938,2.497792,56.978838,5.832165,34.167550,6.119219,6.459967,3.572020,0.000431,0.015467
std,19.274623,12.580783,1.343330,0.341178,0.851666,4.739637,4.205445,1.733862,5.070714,1.620603,...,0.337747,2.095943,7.924475,1.565279,7.411552,1.567884,17.190085,9.497589,0.002117,0.038316
min,0.000000,0.000000,0.100000,0.150000,0.080000,4.540000,2.940000,1.380000,3.120000,1.400000,...,0.000000,0.000000,28.000000,0.600000,5.700000,1.710000,0.000000,0.000000,0.000000,0.000000
25%,112.000000,68.000000,0.880000,1.160000,2.180000,11.200000,27.000000,8.000000,42.000000,8.000000,...,0.300000,1.200000,51.800000,4.800000,29.200000,5.050000,0.000000,0.000000,0.000000,0.000000
50%,124.000000,76.000000,1.270000,1.350000,2.690000,16.000000,29.000000,9.000000,45.000000,8.500000,...,0.500000,1.950000,57.100000,5.600000,34.000000,5.890000,0.010000,0.100000,0.000000,0.000000
75%,137.000000,85.000000,1.910000,1.600000,3.260000,18.700000,32.000000,10.000000,48.000000,9.000000,...,0.700000,3.100000,62.200000,6.700000,38.900000,6.960000,0.010000,0.200000,0.000000,0.000000
max,252.000000,148.000000,28.800000,4.780000,11.500000,39.600000,54.000000,43.000000,68.000000,50.000000,...,2.800000,40.500000,89.000000,15.900000,64.600000,19.500000,105.000000,99.000000,0.020000,0.400000


In [39]:
filtered_train.shape

(38187, 157)

In [44]:
test

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0101,0102,0104,...,979026,979027,A201,A202,A301,A302,A601,A701,A703,A705
0,001f,NaN,NaN,NaN,NaN,NaN,NaN,甲状腺形,甲状腺彩,nan,...,NaN,NaN,详见纸质,胸部CT,nan,nan,详见纸质,6.4,nan,数字化肝
1,0029,NaN,NaN,NaN,NaN,NaN,12.80,甲状腺形,右肾:右,nan,...,NaN,NaN,肺窗示：,肺纤维灶,nan,nan,nan,nan,nan,nan
2,002f,NaN,NaN,NaN,NaN,NaN,NaN,甲状腺形,肝、胆、,nan,...,NaN,NaN,nan,nan,nan,nan,nan,6.8,227.,数字化肝
3,0037,NaN,NaN,NaN,NaN,NaN,NaN,右侧乳腺,右侧乳腺,nan,...,NaN,NaN,胸廓对称,胸部CT,nan,nan,详见纸质,nan,nan,nan
4,0039,NaN,NaN,NaN,NaN,NaN,NaN,双侧乳腺,双侧乳腺,nan,...,NaN,NaN,胸廓对称,CT检查,nan,nan,nan,nan,nan,nan
5,0043,NaN,NaN,NaN,NaN,NaN,NaN,双侧甲状,肝、胆、,nan,...,NaN,NaN,nan,nan,详见纸质,详见纸质,nan,12.1,215.,经数字化
6,0047,NaN,NaN,NaN,NaN,NaN,NaN,nan,前列腺:,nan,...,NaN,NaN,两侧大脑,详见纸质,nan,nan,详见纸质,nan,nan,nan
7,0048,NaN,NaN,NaN,NaN,NaN,NaN,双侧甲状,甲状腺左,nan,...,NaN,NaN,胸廓对称,胸部CT,nan,nan,nan,nan,nan,nan
8,0050,NaN,NaN,NaN,NaN,NaN,NaN,双侧甲状,甲状腺双,nan,...,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan
9,0051,NaN,NaN,NaN,NaN,NaN,NaN,甲状腺形,肝:脂肪,nan,...,NaN,NaN,nan,nan,nan,nan,详见纸质,4.4,265.,脂肪衰减


In [45]:
l4=filtered_train.columns

useful_test=test[l4]

# useful_test=useful_test.drop('Unnamed: 0', axis=1)
useful_test

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0105,0106,0108,0109,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.30,0.50,52.2,5.00,42.0,6.60,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,12.80,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
test_pred1=useful_test.iloc[:, 0:5]
test_fea1=useful_test.iloc[:, 5:]
test_fea1

,004997,0105,0106,0108,0109,100005,100007,100012,100013,10003,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.62,26.7,...,0.30,0.50,52.2,5.00,42.0,6.60,NaN,NaN,NaN,NaN
1,12.80,NaN,NaN,NaN,NaN,NaN,NaN,136.0,NaN,26.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,0.20,155.0,5.20,28.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,13.8,0.12,NaN,5.67,28.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,0.22,96.0,5.15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,12.3,0.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
import pandas as pd

import numpy as np

import seaborn as sns

from pylab import mpl

import lightgbm as lgb

import xgboost as xgb

from sklearn import preprocessing

from sklearn import svm 

from sklearn import cross_validation

from sklearn.cross_validation import KFold

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error




C:\Users\lk\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [48]:
from math import log10,pow
# 计算误差损失（评估分数）
def calc_logloss(true_df,pred_df):
    loss_sum=0
    rows=true_df.shape[0]
    for c in true_df.columns:
        #预测结果必须要>0,否则log函数会报错，导致最终提交结果没有分数
        true_df[c]=true_df[c].apply(lambda x:log10(x+1))
        pred_df[c]=pred_df[c].apply(lambda x:log10(x+1))
        true_df[c+'new']=true_df[c]-pred_df[c]
        true_df[c+'new']=true_df[c+'new'].apply(lambda x:pow(x,2))
        loss_item=(true_df[c+'new'].sum())/rows
        loss_sum+=loss_item
        print('%s的loss：%f'%(c,loss_item))
    print('总的loss：',loss_sum)

In [49]:
X=filtered_train.iloc[:, 5:]
X.describe()

,004997,0105,0106,0108,0109,100005,100007,100012,100013,10003,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
count,3334.000000,1726.000000,1718.000000,1634.000000,1664.000000,12893.000000,15037.000000,4716.000000,4525.000000,18399.000000,...,6762.000000,6762.000000,6866.000000,6867.000000,6866.000000,6866.000000,2720.000000,2609.000000,2250.000000,2250.000000
mean,15.408500,29.641686,8.938882,44.859969,8.605938,18.972380,0.685127,138.394890,4.343708,28.604185,...,0.547938,2.497792,56.978838,5.832165,34.167550,6.119219,6.459967,3.572020,0.000431,0.015467
std,4.739637,4.205445,1.733862,5.070714,1.620603,12.075733,4.188304,29.312641,1.773429,4.207588,...,0.337747,2.095943,7.924475,1.565279,7.411552,1.567884,17.190085,9.497589,0.002117,0.038316
min,4.540000,2.940000,1.380000,3.120000,1.400000,4.000000,0.000000,0.000000,0.570000,7.200000,...,0.000000,0.000000,28.000000,0.600000,5.700000,1.710000,0.000000,0.000000,0.000000,0.000000
25%,11.200000,27.000000,8.000000,42.000000,8.000000,12.100000,0.150000,119.000000,3.550000,25.700000,...,0.300000,1.200000,51.800000,4.800000,29.200000,5.050000,0.000000,0.000000,0.000000,0.000000
50%,16.000000,29.000000,9.000000,45.000000,8.500000,13.000000,0.200000,135.000000,4.420000,28.500000,...,0.500000,1.950000,57.100000,5.600000,34.000000,5.890000,0.010000,0.100000,0.000000,0.000000
75%,18.700000,32.000000,10.000000,48.000000,9.000000,15.000000,0.250000,154.250000,5.060000,31.300000,...,0.700000,3.100000,62.200000,6.700000,38.900000,6.960000,0.010000,0.200000,0.000000,0.000000
max,39.600000,54.000000,43.000000,68.000000,50.000000,56.800000,51.800000,623.000000,76.400000,79.700000,...,2.800000,40.500000,89.000000,15.900000,64.600000,19.500000,105.000000,99.000000,0.020000,0.400000


In [50]:
y=filtered_train.iloc[:, 0:5]

In [51]:
filtered_train

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白,004997,0105,0106,0108,0109,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
0,165.0,100.0,2.08,1.29,3.24,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141.0,97.0,2.64,1.36,4.75,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,120.0,80.0,1.37,1.25,2.66,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100.0,70.0,1.27,2.21,1.73,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110.0,80.0,0.80,1.87,2.21,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,110.0,80.0,1.40,1.74,2.09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,130.0,78.0,2.50,1.22,4.53,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,111.0,71.0,3.36,1.08,2.88,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,94.0,64.0,0.62,1.34,1.67,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,164.0,98.0,1.12,1.51,2.79,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
'''
用线性回归, use linear regression
'''

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X=filtered_train.iloc[:, 5:]
y=filtered_train.iloc[:, 0:5]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
# y_train['收缩压']


linreg = LinearRegression()

# linreg.fit(X_train,y_train['收缩压'])
linreg.fit(X_train,y_train)

# import xgboost as xgb  
# gbm = xgb.XGBRegressor().fit(data.ix[:,25:], data['y'])  
# predictions = gbm.predict(data.ix[:,25:])  
# actuals = data['y']  
# print(mean_squared_error(actuals, predictions))  

y_pred=linreg.predict(X_test)
y_pred=np.ndarray.round(y_pred,3)#保留三位小数
y_test=pd.DataFrame(y_test,columns=['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白'])
y_pred_res=pd.DataFrame(y_pred,columns=['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白'])
# y_pred_res=y_pred_res[]

collumns=y_pred_res.columns
y_test[columns[5]]
y_test=y_test[y_test[columns[5]]>0]

calc_logloss(y_test,y_pred_res)

'\n用线性回归\n'

In [55]:
test_pred1=useful_test.iloc[:, 0:5]
test_fea1=useful_test.iloc[:, 5:]

"\nfor i in range(9538):\n    if (isinstance(test_fea1.loc[i, '809004'], str)):\n        if test_fea1.loc[i, '809004']=='5.22 5.22':\n            test_fea1.loc[i, '809004']=5.22\n        test_fea1.loc[i, '809004']=float(test_fea1.loc[i, '809004'])\n\nfor i in range(9538):\n    if (isinstance(test_fea1.loc[i, '809007'], str)):\n        if test_fea1.loc[i, '809007']=='10.24 8.26':\n            test_fea1.loc[i, '809007']=9.25\n        test_fea1.loc[i, '809007']=float(test_fea1.loc[i, '809007'])\n#         df['涨跌幅'] = df['涨跌幅'].astype(np.float64)\n"

In [59]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X=filtered_train.iloc[:, 5:]
y=filtered_train.iloc[:, 0:5]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=12)


# test_fea1=test_fea1.drop(['Unnamed: 0'], axis=1)

# model = xgb.XGBRegressor(max_depth=5, learning_rate=0.05, n_estimators=160, silent=True, objective='reg:gamma')
model = xgb.XGBRegressor(max_depth=6, learning_rate=0.05, n_estimators=250, silent=True, objective='reg:gamma')
model.fit(X_train, y_train['收缩压'])



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=250,
       n_jobs=1, nthread=None, objective='reg:gamma', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [60]:
# 对测试集进行预测
ans_1 = model.predict(test_fea1)
df_ans_1=pd.DataFrame(ans_1)
print('--------done')
df_ans_1

--------done


,0
0,124.464363
1,118.226814
2,136.859436
3,133.412521
4,125.593231
5,113.321228
6,124.970329
7,132.972519
8,134.373123
9,128.856369


In [72]:
X_test

,004997,0105,0106,0108,0109,100005,100007,100012,100013,10003,...,979018,979019,979020,979021,979022,979023,979024,979025,979026,979027
29674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.7,...,0.70,5.50,61.0,4.10,28.7,8.35,0.00,0.0,0.00,0.0
18372,NaN,NaN,NaN,NaN,NaN,17.8,0.13,NaN,NaN,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.7,...,0.60,2.50,56.1,5.90,34.9,5.85,0.00,0.1,0.00,0.0
17503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5593,NaN,NaN,NaN,NaN,NaN,13.7,0.21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23401,16.0,NaN,NaN,NaN,NaN,12.0,0.12,139.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33649,NaN,NaN,NaN,NaN,NaN,12.1,0.21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31533,NaN,NaN,NaN,NaN,NaN,14.6,0.22,NaN,NaN,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
y_test

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
29674,118.0,78.0,0.93,1.59,3.70
18372,135.0,90.0,2.20,1.36,3.09
32156,110.0,71.0,3.31,1.24,3.30
21011,135.0,74.0,1.83,0.77,2.05
17503,110.0,66.0,0.56,1.07,2.16
5593,140.0,80.0,0.92,1.34,1.53
23401,122.0,87.0,2.17,1.74,3.59
33649,106.0,60.0,0.63,1.56,3.04
17963,123.0,79.0,3.12,1.06,2.21
31533,138.0,88.0,1.23,1.93,3.13


In [61]:
y_train

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
18340,104.0,68.0,0.90,1.50,3.20
17818,129.0,72.0,1.60,1.41,3.29
10398,140.0,90.0,1.25,1.34,2.90
10550,102.0,72.0,0.97,1.00,2.45
29708,112.0,64.0,0.94,1.59,2.84
35508,167.0,84.0,0.97,1.92,2.64
25723,114.0,86.0,1.59,1.27,2.17
22559,113.0,77.0,0.98,1.57,2.13
30511,155.0,95.0,0.78,1.15,2.02
19345,132.0,85.0,2.17,1.02,3.74


In [62]:
# model2 = xgb.XGBRegressor(max_depth=5, learning_rate=0.1, n_estimators=160, silent=True, objective='reg:gamma')
model2 = xgb.XGBRegressor(max_depth=6, learning_rate=0.05, n_estimators=250, silent=True, objective='reg:gamma')
model2.fit(X_train, y_train['舒张压'])
print('-----')

model3 = xgb.XGBRegressor(max_depth=6, learning_rate=0.05, n_estimators=250, silent=True, objective='reg:gamma')
model3.fit(X_train, y_train['血清甘油三酯'])
print('-----')

model4 = xgb.XGBRegressor(max_depth=6, learning_rate=0.05, n_estimators=250, silent=True, objective='reg:gamma')
model4.fit(X_train, y_train['血清高密度脂蛋白'])
print('-----')

model5 = xgb.XGBRegressor(max_depth=6, learning_rate=0.05, n_estimators=250, silent=True, objective='reg:gamma')
model5.fit(X_train, y_train['血清低密度脂蛋白'])
print('-----')

-----
-----
-----
-----


In [63]:
# 对测试集进行预测
ans_1 = model.predict(test_fea1)
ans_2 = model2.predict(test_fea1)
ans_3 = model3.predict(test_fea1)
ans_4 = model4.predict(test_fea1)
ans_5 = model5.predict(test_fea1)

df_ans_1=pd.DataFrame(ans_1)
print('--------done')
# df_ans_1


# ans_2 = model.predict(test_fea1)
df_ans_2=pd.DataFrame(ans_2)
print('--------done')
# df_ans_1

# ans_3 = model.predict(test_fea1)
df_ans_3=pd.DataFrame(ans_3)
print('--------done')

# ans_4 = model.predict(test_fea1)
df_ans_4=pd.DataFrame(ans_4)
print('--------done')

# ans_5 = model.predict(test_fea1)
df_ans_5=pd.DataFrame(ans_5)
print('--------done')

--------done
--------done
--------done
--------done
--------done


In [64]:
df_ans_1

,0
0,124.464363
1,118.226814
2,136.859436
3,133.412521
4,125.593231
5,113.321228
6,124.970329
7,132.972519
8,134.373123
9,128.856369


In [65]:
df_ans_2

,0
0,78.745934
1,69.465714
2,90.468872
3,80.012329
4,72.261818
5,71.223946
6,78.007118
7,82.452858
8,87.475754
9,78.660522


In [66]:
df_ans_3

,0
0,1.625024
1,0.932204
2,3.344614
3,1.464233
4,1.544936
5,1.416875
6,1.377613
7,2.256686
8,2.191610
9,1.568626


In [67]:
df_ans_4

,0
0,1.459283
1,1.690745
2,1.225058
3,1.591745
4,1.506686
5,1.694637
6,1.661030
7,1.278102
8,1.337044
9,1.437953


In [68]:
df_ans_5[0]

0       2.855099
1       2.872324
2       2.923348
3       1.974513
4       3.162366
5       2.938957
6       2.173648
7       2.674551
8       2.709219
9       2.698594
10      2.752852
11      2.568740
12      3.120621
13      3.110864
14      2.768070
15      2.723458
16      2.339510
17      4.183207
18      2.367263
19      2.733211
20      2.304685
21      2.659625
22      2.259782
23      2.452933
24      2.896603
25      3.131210
26      3.187168
27      2.812501
28      2.133736
29      2.518029
          ...   
9502    2.935027
9503    2.081518
9504    2.514792
9505    2.753090
9506    2.189802
9507    2.974874
9508    3.013799
9509    2.535927
9510    2.606968
9511    2.326622
9512    2.731845
9513    3.407140
9514    3.635228
9515    2.188248
9516    2.847490
9517    2.647397
9518    3.362881
9519    3.810146
9520    2.900802
9521    2.538463
9522    2.981851
9523    3.005247
9524    2.378846
9525    2.312267
9526    2.426083
9527    2.837439
9528    2.798510
9529    3.1428

In [69]:
df_ans_5

,0
0,2.855099
1,2.872324
2,2.923348
3,1.974513
4,3.162366
5,2.938957
6,2.173648
7,2.674551
8,2.709219
9,2.698594


In [70]:
test_v2=pd.read_csv('D:\\1python notes\\aliyun\\new\\meinian_round1_test_b_20180505.csv',sep=',',encoding='gbk')

# test_v1


test_v2['收缩压']=df_ans_1[0]
test_v2['舒张压']=df_ans_2[0]
test_v2['血清甘油三酯']=df_ans_3[0]
test_v2['血清高密度脂蛋白']=df_ans_4[0]
test_v2['血清低密度脂蛋白']=df_ans_5[0]

test_v2.to_csv('D:\\1python notes\\aliyun\\new\\temp\\test505.csv',index=False,header=None,encoding='utf-8')
test_v2

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
0,001f849a29f618fa98b23289682d7b37,124.464363,78.745934,1.625024,1.459283,2.855099
1,0029a7c3e6d454b22c9b1d5233defc2b,118.226814,69.465714,0.932204,1.690745,2.872324
2,002fe4622e31ccc987a770ca02cdc812,136.859436,90.468872,3.344614,1.225058,2.923348
3,00372d4cde397f592b0004ebb5a25a2f,133.412521,80.012329,1.464233,1.591745,1.974513
4,003955abc4de39314f4f872351689046,125.593231,72.261818,1.544936,1.506686,3.162366
5,004304681ae963ee9369fdf7dc2e94e7,113.321228,71.223946,1.416875,1.694637,2.938957
6,0047a2cf729c5c19c43397770c32d87d,124.970329,78.007118,1.377613,1.661030,2.173648
7,0048f6ad7ac877f42d564c42eb725e3a,132.972519,82.452858,2.256686,1.278102,2.674551
8,005062ba9b098dcd0ebcfa3b881a8a4f,134.373123,87.475754,2.191610,1.337044,2.709219
9,005168460d7261ac881b404684e1379d,128.856369,78.660522,1.568626,1.437953,2.698594
